<a href="https://colab.research.google.com/github/PanoEvJ/summarization_RLHF/blob/main/rlhf_PPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U git+https://github.com/lvwerra/trl.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U git+https://github.com/huggingface/peft.git

!pip install -q transformers==4.30
!pip install -q -U sentencepiece
!pip install -q huggingface_hub
!pip install -q tdqm torch>=0.3.0
!pip install -q -U bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import os
import random
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset, Dataset
from peft import AutoPeftModelForCausalLM, LoraConfig, PeftConfig, PeftModel
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments

from trl import SFTTrainer
from trl.trainer import ConstantLengthDataset

In [4]:
from accelerate import Accelerator
from transformers import Adafactor, pipeline

from trl import PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler

In [ ]:
from datasets import get_dataset_split_names

dataset_name = 'CarperAI/openai_summarize_comparisons'
get_dataset_split_names(dataset_name)

dataset = load_dataset('CarperAI/openai_summarize_comparisons', split='train').shuffle(seed=42)
ppo_dataset = Dataset(dataset['prompt'])

In [17]:
len(dataset)

92534

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_use_double_quant=True,
    bnb_8bit_quant_type="nf4",
    bnb_8bit_compute_dtype=torch.bfloat16,
)

In [21]:
ref_model_id = 't5-base'
policy_model_id = 'PanoEvJ/T5_base_SFT_summarization'

ref_model = AutoModelForSeq2SeqLM.from_pretrained(ref_model_id,
                                                  device_map="auto",
                                                  # load_in_8bit=True,
                                                  quantization_config=bnb_config,
                                                 )
policy_model = AutoModelForSeq2SeqLM.from_pretrained(policy_model_id,
                                                     device_map="auto",
                                                    #  load_in_8bit=True,
                                                     quantization_config=bnb_config
                                                    )

In [23]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print_trainable_parameters(ref_model)

trainable params: 81345792 || all params: 222903552 || trainable%: 36.49371724682072


In [ ]:
# see the available modules by printint out the model
print(policy_model)

In [20]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q", "k", "v"],
    bias="none",
    task_type="SEQ_2_SEQ_LM",
)

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("./rewards_model/")
tokenizer = AutoTokenizer.from_pretrained("./rewards_model/")

NameError: ignored